In [ ]:
%load_ext autoreload
%autoreload 2

In [15]:
import torch

print(f"PyTorch has version {torch.__version__} with cuda {torch.version.cuda}")

import reading_utils

PyTorch has version 2.0.0+cu117 with cuda 11.7


In [7]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1w-aWuQJEr2BqDkEMQdcKnRBBJapwLtBT")

Retrieving folder list


Processing file 1zz-pG1qkf6oAZ2R39_Xv1tDauq53XkXr metadata.json
Processing file 1Fi0GueqwzabQrNTVhUgnk3vKVIBY0fE9 test.pt
Processing file 1mqjL0BcOpvMfri1MdmOuRUXGfvwMoKoP train.pt
Processing file 1gtUg0L2EgxdYl7fa-wecE6qGI5xriySz valid.pt
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1zz-pG1qkf6oAZ2R39_Xv1tDauq53XkXr
To: d:\UCB\Sp2023\CS282A\cs-182-project\Sand\metadata.json
100%|██████████| 360/360 [00:00<?, ?B/s] 
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Fi0GueqwzabQrNTVhUgnk3vKVIBY0fE9
From (redirected): https://drive.google.com/uc?id=1Fi0GueqwzabQrNTVhUgnk3vKVIBY0fE9&confirm=t&uuid=d69adc12-d235-453a-9e8d-66c66c5c2245
To: d:\UCB\Sp2023\CS282A\cs-182-project\Sand\test.pt
100%|██████████| 98.2M/98.2M [00:01<00:00, 87.3MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1mqjL0BcOpvMfri1MdmOuRUXGfvwMoKoP
From (redirected): https://drive.google.com/uc?id=1mqjL0BcOpvMfri1MdmOuRUXGfvwMoKoP&confirm=t&uuid=bec4a650-b9d6-41d9-b2c9-f4007bc951d0
To: d:\UCB\Sp2023\CS282A\cs-182-project\Sand\train.pt
100%|██████████| 3.07G/3.07G [00:32<00:00, 94.5MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1gtUg0L2E

['d:\\UCB\\Sp2023\\CS282A\\cs-182-project\\Sand\\metadata.json',
 'd:\\UCB\\Sp2023\\CS282A\\cs-182-project\\Sand\\test.pt',
 'd:\\UCB\\Sp2023\\CS282A\\cs-182-project\\Sand\\train.pt',
 'd:\\UCB\\Sp2023\\CS282A\\cs-182-project\\Sand\\valid.pt']

In [13]:
ds_train = torch.load('Sand/train.pt')
ds_test = torch.load('Sand/test.pt')
ds_valid = torch.load('Sand/valid.pt')

{'particle_type': tensor([6, 6, 6,  ..., 6, 6, 6]),
 'key': <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 'position': tensor([[[0.4210, 0.5497],
          [0.4064, 0.5722],
          [0.3917, 0.5945],
          ...,
          [0.5590, 0.1381],
          [0.5590, 0.1381],
          [0.5590, 0.1381]],
 
         [[0.4105, 0.5519],
          [0.3959, 0.5744],
          [0.3813, 0.5967],
          ...,
          [0.5227, 0.1448],
          [0.5227, 0.1448],
          [0.5227, 0.1448]],
 
         [[0.4101, 0.5444],
          [0.3955, 0.5670],
          [0.3809, 0.5893],
          ...,
          [0.5404, 0.1396],
          [0.5404, 0.1396],
          [0.5404, 0.1397]],
 
         ...,
 
         [[0.6467, 0.6702],
          [0.6564, 0.6687],
          [0.6660, 0.6671],
          ...,
          [0.7642, 0.2511],
          [0.7642, 0.2511],
          [0.7642, 0.2511]],
 
         [[0.8146, 0.6852],
          [0.8243, 0.6838],
          [0.8339, 0.6822],
          ...,
          [0.8362, 0.381